Let's first define available tokens

In [64]:
from enum import Enum

class Token(Enum):
    EOI       = 0  # End of input
    SEMI      = 1  # ;
    PLUS      = 2  # +
    TIMES     = 3  # *
    LP        = 4  # (
    RP        = 5  # )
    NUM_OR_ID = 6  # decimal number or identifier

Now we want to define our expressions

In [65]:
expressions = """
2 + 2
14 * 5
(10 * 2) + 5

60 * 70
"""

Ok, so now we will be doing lookahead with expressions[0] and if given character was analyzed we will be doing pop(0)

In [66]:
chars = list(expressions)
print(chars)

['\n', '2', ' ', '+', ' ', '2', '\n', '1', '4', ' ', '*', ' ', '5', '\n', '(', '1', '0', ' ', '*', ' ', '2', ')', ' ', '+', ' ', '5', '\n', '\n', '6', '0', ' ', '*', ' ', '7', '0', '\n']


Below we define useful mapping between symbol and token.

In [67]:
character_to_token = {
    ';': Token.SEMI,
    '+': Token.PLUS,
    '*': Token.TIMES,
    '(': Token.LP,
    ')': Token.RP
}

In [68]:
from collections import namedtuple
Lexeme = namedtuple('Lexeme', ['type', 'value'])

In [69]:
def lex(chars):
    if len(chars) == 0:
        return Lexeme(Token.EOI, None)

    current = chars.pop(0)
    
    while 1:
        while len(chars) > 0 and current == ' ':
            current = chars.pop(0)
        
        while len(chars) > 0:
            # Get the next token
            if current in character_to_token:
                return Lexeme(character_to_token[current], None)
            elif current in ('\n', '\t', ' '):
                return None
            else:
                if not current.isalnum():
                    print(f"Ignoring illegal input {current}")
                    return None
                else:
                    value = [current]
                    while len(chars) > 0 and chars[0].isalnum():
                        value.append(chars.pop(0))
                    return Lexeme(Token.NUM_OR_ID, value)

        if len(chars) == 0:
            return Lexeme(Token.EOI, None)

    return Lexeme(Token.EOI, None)

In [70]:
lexeme = lex(chars)

def proceed(lexeme):
    if lexeme == None:
        return True
    elif lexeme.type == Token.EOI:
        return False
    return True

while proceed(lexeme):
    print(lexeme)
    lexeme = lex(chars)

print(lexeme)

None
Lexeme(type=<Token.NUM_OR_ID: 6>, value=['2'])
Lexeme(type=<Token.PLUS: 2>, value=None)
Lexeme(type=<Token.NUM_OR_ID: 6>, value=['2'])
None
Lexeme(type=<Token.NUM_OR_ID: 6>, value=['1', '4'])
Lexeme(type=<Token.TIMES: 3>, value=None)
Lexeme(type=<Token.NUM_OR_ID: 6>, value=['5'])
None
Lexeme(type=<Token.LP: 4>, value=None)
Lexeme(type=<Token.NUM_OR_ID: 6>, value=['1', '0'])
Lexeme(type=<Token.TIMES: 3>, value=None)
Lexeme(type=<Token.NUM_OR_ID: 6>, value=['2'])
Lexeme(type=<Token.RP: 5>, value=None)
Lexeme(type=<Token.PLUS: 2>, value=None)
Lexeme(type=<Token.NUM_OR_ID: 6>, value=['5'])
None
None
Lexeme(type=<Token.NUM_OR_ID: 6>, value=['6', '0'])
Lexeme(type=<Token.TIMES: 3>, value=None)
Lexeme(type=<Token.NUM_OR_ID: 6>, value=['7', '0'])
Lexeme(type=<Token.EOI: 0>, value=None)


In [71]:
expressions = """
2 + 2
14 * 5
(10 * 2) + 5

60 * 70
"""

In [72]:
chars = list(expressions)
print(chars)

['\n', '2', ' ', '+', ' ', '2', '\n', '1', '4', ' ', '*', ' ', '5', '\n', '(', '1', '0', ' ', '*', ' ', '2', ')', ' ', '+', ' ', '5', '\n', '\n', '6', '0', ' ', '*', ' ', '7', '0', '\n']


In [123]:
class Lexer(object):
    def __init__(self, chars):
        self.lookahead = None
        self.chars = chars

    def match(self, token):
        # Return true if "token" matches the current lookahead symbol.
        if not self.lookahead:
            while not self.lookahead:
                self.lookahead = lex(self.chars)
                print(f"New lookahead: {self.lookahead}")

        return token == self.lookahead.type

    def advance(self):
        # Advance the lookahead to the next input symbol.
        self.lookahead = lex(self.chars)
        print(f"Changed lookahead: {self.lookahead}")

In [86]:
lexer = Lexer(chars)
lexer.match(Token.PLUS)

lexer.match(Token.PLUS)

New lookahead: Lexeme(type=<Token.NUM_OR_ID: 6>, value=['2'])


False

In [75]:
lexer.advance()

Changed lookahead: Lexeme(type=<Token.NUM_OR_ID: 6>, value=['2'])


In [76]:
print(chars)

[' ', '+', ' ', '2', '\n', '1', '4', ' ', '*', ' ', '5', '\n', '(', '1', '0', ' ', '*', ' ', '2', ')', ' ', '+', ' ', '5', '\n', '\n', '6', '0', ' ', '*', ' ', '7', '0', '\n']


In [129]:
class ParserException(Exception):
    """Parser exception."""

class Parser(object):
    def __init__(self, lexer):
        self.lexer = lexer

    def factor(self):
        """
        factor -> NUM_OR_ID
               |  LP expression RP
        """
        print(f"factor {self.lexer.chars}")
        if self.lexer.match(Token.NUM_OR_ID):
            self.lexer.advance()
        elif self.lexer.match(Token.LP):
            self.lexer.advance()
            self.expression()
            if self.lexer.match(Token.RP):
                self.lexer.advance()
            else:
                raise ParserException("Mismatch parenthesis LINENO (TODO)")
        else:
            raise ParserException("Number or identifier expected LINENO (TODO)")

    def term_prime(self):
        """
        term' -> TIMES factor term'
              |  epsilon
        """
        print(f"term_prime {self.lexer.chars}")
        if self.lexer.match(Token.TIMES):
            self.lexer.advance()
            self.factor()
            self.term_prime()

    def term(self):
        """
        term -> factor term'
        """
        print(f"term {self.lexer.chars}")
        self.factor()
        self.term_prime()

    def expr_prime(self):
        """
        expression' -> PLUS term expression'
                     | epsilon
        """
        print(f"expr_prime {self.lexer.chars}")
        if self.lexer.match(Token.PLUS):
            self.lexer.advance()
            self.term()
            self.expr_prime()

    def expression(self):
        """
        expression -> term expression'
        """
        print(f"expression {self.lexer.chars}")
        self.term()
        self.expr_prime()

    def statements(self):
        """
        statements -> expression SEMI
                |     expression SEMI statement
        """
        print(f"statements {self.lexer.chars}")
        self.expression()

        if self.lexer.match(Token.SEMI):
            self.lexer.advance()
        else:
            print("Inserting missing semicolon: LINENO (TODO)")

        if (not self.lexer.match(Token.EOI)):
            self.statements()

In [136]:
expressions = """
2 * 100;
"""

chars = list(expressions)
print(chars)

['\n', '2', ' ', '*', ' ', '1', '0', '0', ';', '\n']


In [137]:
lexer = Lexer(chars)

parser = Parser(lexer)

# TODO: parsing statements with single expression works fine
#       but once more expressions are present it goes into
#       infinit loop somewhere.
parser.statements()

statements ['\n', '2', ' ', '*', ' ', '1', '0', '0', ';', '\n']
expression ['\n', '2', ' ', '*', ' ', '1', '0', '0', ';', '\n']
term ['\n', '2', ' ', '*', ' ', '1', '0', '0', ';', '\n']
factor ['\n', '2', ' ', '*', ' ', '1', '0', '0', ';', '\n']
New lookahead: None
New lookahead: Lexeme(type=<Token.NUM_OR_ID: 6>, value=['2'])
Changed lookahead: Lexeme(type=<Token.TIMES: 3>, value=None)
term_prime [' ', '1', '0', '0', ';', '\n']
Changed lookahead: Lexeme(type=<Token.NUM_OR_ID: 6>, value=['1', '0', '0'])
factor [';', '\n']
Changed lookahead: Lexeme(type=<Token.SEMI: 1>, value=None)
term_prime ['\n']
expr_prime ['\n']
Changed lookahead: Lexeme(type=<Token.EOI: 0>, value=None)
